# Symbolic Differentiation vs Automatic Differentiation

Consider the function below that, at least computationally, is very simple.

In [49]:
def func(x):
    y = x
    for i in range(10):
        y = sin(x + y)

    return y

We can compute a derivative symbolically, but it is of course horrendous (see below).  Think of how much worse it would be if we chose a function with products, and/or iterated more than 10 times.

In [50]:
from sympy import diff, Symbol, sin
from __future__ import print_function

x = Symbol('x')
dexp = diff(func(x), x)
print(dexp)

(((((((((2*cos(2*x) + 1)*cos(x + sin(2*x)) + 1)*cos(x + sin(x + sin(2*x))) + 1)*cos(x + sin(x + sin(x + sin(2*x)))) + 1)*cos(x + sin(x + sin(x + sin(x + sin(2*x))))) + 1)*cos(x + sin(x + sin(x + sin(x + sin(x + sin(2*x)))))) + 1)*cos(x + sin(x + sin(x + sin(x + sin(x + sin(x + sin(2*x))))))) + 1)*cos(x + sin(x + sin(x + sin(x + sin(x + sin(x + sin(x + sin(2*x)))))))) + 1)*cos(x + sin(x + sin(x + sin(x + sin(x + sin(x + sin(x + sin(x + sin(2*x))))))))) + 1)*cos(x + sin(x + sin(x + sin(x + sin(x + sin(x + sin(x + sin(x + sin(x + sin(2*x))))))))))


Even if we could somehow symbolically compute derivatives for our simulations, the result is terribly slow.  Let's evaluate this function 10 times and time how long it takes.  Of course in an optimization we would need to evaluate the gradient way more than 10 times, but I don't want to wait that long. We will just evaluate at the same point all 10 times because it doesn't really matter. Notice I'm not even timing the time it takes to generate the expression, we are assuming that is free.  

In [51]:
import time
repeat = 10

start_time = time.time()

for i in range(repeat):
    dfdx = dexp.evalf(subs={x: 0.1})

t1 = time.time() - start_time

print('time =', t1)
print('dfdx =', dfdx)

time = 7.98072695732
dfdx = 3.04145180189667


Let's compare with automatic differentiation, running at the same point 10 times as well.

In [52]:
from algopy import UTPM, sin

start_time = time.time()

for i in range(repeat):
    x_algopy = UTPM.init_jacobian(0.1)
    y_algopy = func(x_algopy)
    dfdx = UTPM.extract_jacobian(y_algopy)
    
t2 = time.time() - start_time

print('time =', t2)
print('dfdx =', dfdx)

time = 0.0112597942352
dfdx = [ 3.0414518]


The ratio of time for symbolic differentiation as compared to automatic differentiation is:

In [53]:
print('time ratio:', t1/t2)

time ratio: 708.780888051


Note, it takes about 3 orders of magnitude more time to evaluate the derivatives of this function with a symbolic expression.  Imagine how much worse it would be with multiple dimensions, or more complex functions.  By contract, automatic differentiation scales well with a processing speed on the order of the original function call (assuming source code transformation, it's a bit slower for operator overloading as is used here).